In [1]:
import numpy as np

In [4]:
s = np.random.uniform(low = -2.0, high = 2.0, size = 10000)

In [7]:
t = np.random.uniform(low = 0, high = 5.0, size = 10000)

In [12]:
S = np.vstack((s,t)).T

In [14]:
u = -2*s-0.2*np.sin(0.3*t*s)*s

In [18]:
import torch 
import torch.nn as nn
from torchsummary import summary

In [48]:
class Neural(nn.Module):
    def __init__(self):
        super(Neural,self).__init__()
        self.fc1 = nn.Linear(2,8)
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(8,1,bias=False)
    def forward(self,x):
        x = self.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

model = Neural()
summary(model,(1,2))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 8]              24
              Tanh-2                 [-1, 1, 8]               0
            Linear-3                 [-1, 1, 1]               8
Total params: 32
Trainable params: 32
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [49]:
from torch.utils.data import DataLoader,TensorDataset

In [50]:
dataset = DataLoader(TensorDataset(torch.Tensor(S),torch.Tensor(u)),batch_size = 1, shuffle = True)

In [51]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [52]:
epochs = 20
for epoch in range(epochs):
    train_loss = 0
    model.train()
    for idx, inp in enumerate(dataset):
        data,target = inp
        optimizer.zero_grad()
        pred = model(data)
        loss = criterion(target,pred)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"Epoch {epoch+1}: MSE = {train_loss/len(dataset)}")

Epoch 1: MSE = 0.17171661894053555
Epoch 2: MSE = 0.0023836693494926507
Epoch 3: MSE = 0.001617050447203198
Epoch 4: MSE = 0.0014340893954555116
Epoch 5: MSE = 0.0012997531045300728
Epoch 6: MSE = 0.0012287913063229873
Epoch 7: MSE = 0.0011732362303478055
Epoch 8: MSE = 0.0011278445157889045
Epoch 9: MSE = 0.0010645430765723735
Epoch 10: MSE = 0.000969185146115821
Epoch 11: MSE = 0.000992362496416706
Epoch 12: MSE = 0.0008723999736446261
Epoch 13: MSE = 0.0008027411529372248
Epoch 14: MSE = 0.000811734926372287
Epoch 15: MSE = 0.0007640451662981317
Epoch 16: MSE = 0.0007311093594343224
Epoch 17: MSE = 0.0007719620227243303
Epoch 18: MSE = 0.0007288723304066187
Epoch 19: MSE = 0.0007044920111535414
Epoch 20: MSE = 0.0006680309349017564


In [56]:
model.fc1.weight

Parameter containing:
tensor([[ 0.6565, -0.0492],
        [-1.0053,  0.2452],
        [ 0.5934,  0.0217],
        [-0.2612,  0.0655],
        [ 0.4904, -0.0372],
        [ 0.6533,  0.2163],
        [ 0.0286, -0.5248],
        [-0.9361,  0.2955]], requires_grad=True)

In [57]:
model.fc2.weight

Parameter containing:
tensor([[-1.0675,  0.7242, -0.9484,  1.1355, -0.7161, -1.3115,  0.4865,  0.9666]],
       requires_grad=True)

In [58]:
model.fc1.bias

Parameter containing:
tensor([ 1.5045, -0.2904,  0.8204,  0.1544,  1.0847, -1.5262,  0.4379,  1.0963],
       requires_grad=True)